<b>For use with: <font color="red" >Alpaca Kernel</font></b>


# 17B - Testing and Calibration

## Overview

Before you begin assembling your potentiostat, in this assignment, you will learn about the limitations of Alpaca and Pico by measuring noise, resolution, and differences between requested and generated signal.

We expect you to eventually make a voltammogram with positive and negative currents and voltages. The location of the voltammogram peak depends on the substance you measure, but can be influenced by limitations of your system. In this 17B assignment you will explore those limitations.


**If you are still unsure which design to choose, feel free to reach out to the TAs and discuss which option would be the most suitable for you and your partner.**


### Goals

>1. Learn:
>    - how to save and read files with Pico, and make a habit of saving data regularly
>    - how to handle errors effectively
>    - more about the noise in Alpaca
>2. Calibrate your DAC Assistant
>3. Understand the limits of Alpaca+Pico measurements

### Outline 

>1. Implement 1 - Introduction - both designs
>    - I1.1 - Jumpers
>    - I1.2 - Picotools
>    - I1.3 - Saving files
>    - I1.4 - Handling errors
>2. Implement 2 - Double ADC design only - ADCs
>    - link to 19f
>3. Implement 3 - DAC - both designs
>    - I3.1 - Accuracy
>    - I3.2 - Resolution
>4. Implement 4 - Double ADC design only - DAC Assistant
>    - I4.1 Calibration
>    - I4.2 Resolution
>5. Implement 5 - Double ADC design only - GAIN 1:3
>    - I5.1 - Configure GAIN 1:3
>6. Implement 6 Helper functions
>7. Compare and Conclude 



## Implement 1: Introduction

### I1.1: Jumpers

**Check your Alpaca - are all jumpers present?** <br>

The Fritzing just below shows how your Alpaca should look like at the beginning of this project.<br> 
**Make sure that your Alpaca matches this layout.** <br>

**Double check the four jumpers at `SPI DIRECT TO DAQ` and the two jumpers (or wires) at `AMPLIFIER DIRECT TO NANO`**

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/17_Fritzing_Alpaca_Check_Jumpers.png" width=1000>
<br>
<em>Default ALPACA configuration for the Final Project</em>
</div>
<br>

### I1.2: `Picotools` - Module for testing the potentiostat

In the previous assignments, we defined all our custom functions in the cell of the notebook, which makes Jupyter notebooks so handy for prototyping, but for a larger project, this approach creates notebooks that are cumbersome to navigate. So, when your custom function is tested and ready for use later, or simply to have a better overview in your notebook, you may store its definition in an external Python file - a library, which is a simple, and a good practice to improve your workflow.

As an example, we prepared a small library with some familiar and some new functions for automating the testing of the potentiostat, the `picotools.py`. 

> Download this file from Brightspace and save it in the folder with the notebook 17B.

This file has to be sent to Pico using the *Alpaca kernel*, and imported like any other module that you have been using so far. This procedure will be explained throughout this notebook.

**Feel free to take a peek into this file to learn more about the function that we prepared for you to speed up and simplify some of the procedures.**

At some points throughout the Final Project, you might need to tailor some functions for your application. If you want to write your own, improved functions, you may simply add them to `picotools.py`, when they're ready for "deployment". You could also create your own library(-ies) to store some of the advanced Voltammetry procedures of your Final Project. So, consider the use case of *picotools* as a basic example and an inspiration towards advanced programming practices with Python.

> *Please, bear in mind that *Picotools* is currently in version 1.1, so it is not a production-level module. It wasn't thoroughly tested by a large community, like for example `numpy`, so prepare to encounter many constraints, minimal documentation, and potentially some serious bugs. If that happens, please report them to us or take the matter in your hands and improve them right away.*

#### Import `picotools`

Follow the procedure in the cells below to import `picotools`.

> This procedure assumes that you have already downloaded `picotools.py` from Brightspace and placed it in the same folder with this notebook.

In [ ]:
%serialconnect to --port="COM5"

In [ ]:
%sendtofile /picotools.py --source picotools.py

In [ ]:
import picotools as pico

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
import machine

import time

Now, all functions and variables specified in the `picotools.py` are available under as methods of the `pico.` object, so for example:

```python
# Example of a function in picotools
pico.test_ADC()
```

Note that we also have to additionally import all the usual modules.

### I1.3: Saving files

Because of the very limited storage in Pico's memory, you will have to develop a habit to regularly save your measurements. 

It is **crucial** in this project!

Because of the amount of data that will be generated during your measurements, you will very often overload Pico's memory, and most likely have to reset it regularly.
In this notebook, we will show you how to do it safely and to avoid frustration of lost data.

Follow the recipe below to learn how to shuttle files between Pico and your computer. It is a little cumbersome, so we prepared some examples for you.

#### Saving textfiles

In the example below, you can also learn how to make a timestamp for your measurement.

In [ ]:
# Initialize the RTC (Real-time Clock)
rtc = machine.RTC()

# Get the current date and time
year, month, day, weekday, hours, minutes, seconds, subseconds = rtc.datetime()

# Define the file path
file_path = 'Greeting.txt'

# Create the greeting message
file_content = (
    "Hello from Pico Pi and Alpaca!\n"
    f"File saved on: {year:04d}-{month:02d}-{day:02d} {hours:02d}:{minutes:02d}:{seconds:02d}.\n"
)

# Writing to the file
with open(file_path, 'w') as file:
    file.write(file_content)

print("Greeting saved to Greeting.txt")

In [ ]:
# This is the essential command to fetch your file from Pico to your computer
%fetchfile --binary "Greeting.txt" "Greeting_from_Pico.txt"

In [ ]:
# This is a helper function to list all files currently in Pico's storage
pico.list_files()

In [ ]:
# This is how you can remove a file
pico.remove_file('Greeting.txt')

In [ ]:
pico.list_files()

#### Saving numerical data

Once you record the signal from multiple ADCs during a single measurement, **always save it in a temporary array on Pico**, i.e. `temporary_data.npy` and then fetch it from Pico to your computer. 

There is usually only enough storage on Pico for **just one** such aggregated array, so every time you run a new, *long* measurement, you must overwrite the already existing `temporary_data.npy` file. You could also, always remove it after use and create a new one after the next measurement. 

> How long is a *long* measurement? <br>
>
> - Anything with more than `NUM_SAMPLES > 500` might cause some trouble for storage or plotting. Remember that Pico will most likely have to handle multiple arrays of that size internally.
>   <br><br> **For the regular measurements, we recommend `NUM_SAMPLES=1000`** <br><br> and only if you are confident that your code works, go for longer arrays in your final measurement. <br> *In our experience, and with the most optimised code, we managed to succesfully record and handle three arrays of `NUM_SAMPLES=3600`.*

This procedure is again a little cumbersome, but **we will push Pico to its limits in this project to run high quality measurements.**

You may use the code below as a template for later use.

In [ ]:
DATA=np.zeros((3,3))
DATA[0,:]=1 
DATA[1,:]=2
DATA[2,:]=3

np.save('temporary_data.npy', DATA)
del(DATA)

In [ ]:
%fetchfile --binary "temporary_data.npy" "Greeting_DATA_from_Pico.npy"

💡 **Always check if the `Fetched XXX=XXX bytes`.** There are known issues with large files. Retrying often works, but for very large files, different solutions must be found.

In [ ]:
pico.remove_file('temporary_data.npy')

## I1.4:Handling Errors

Saving data and optimising storage is one thing, but Pico's internal memory is another, and managing that is a very difficult task, so in order to prevent trouble here, we propose that you **regularly reset your Pico.** 

> What is the `machine.soft_reset()`?
>
> - In short, *Soft reset* mimics disconnecting and connecting the USB cable between Pico and your computer.
> - It restarts Pico, which means that all your variables, i.e. you unsaved data, will be lost. We perform this reset to easily clear Pico's internal memory (not the storage, not your files). This means that all typical errors with plotting and handling data will be resolved.
> - It also means that you will have to re-import all usual modules.

**Use the two cells below to *restart* your Pico and start anew.**

### Resetting Pico


In [ ]:
# Note that you can execute machine.soft_reset() only when Pico is already connected. 
machine.soft_reset()

In [ ]:
# Default cell for re-connecting Pico, re-importing picotools and the usual modules.
%serialconnect to --port="COM5"

%sendtofile /picotools.py --source picotools.py

import picotools as pico

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# If you want to disconnect Pico from this notebook, use this command:
# %disconnect

## Implement 2: Exploring the limits of the ADCs

Did you know that measuring very low voltages with Pico might be problematic? This is especially relevant for the Double ADC Design. 
Feel free to explore it more deeply in the 19f manual. 

## Implement 3: Exploring the limits of the DAC

In this section, you will have a closer look at the accuracy, noise and the resolution of the DAC. Use the provided Fritzing for this test.

> Goal: Measure the noise in the signal applied from `DACA` to the `ADC`s. <br>
> <br>
> You will be using amplifiers (attenuations 1:1 and 1:3) and measuring directly to Ain2.


<details>
<summary>
<font color='darkred' size=4>💡 <b>Fritzing: DAC Test</b></font>
</summary>
<br>
<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/17_Fritzing_DAC_Test.png" width=1000>
<br>
<em>DAC Test</em>
</div><br>
</details> <br>

Let's again start with a reset to clear Pico's memory.

In [ ]:
machine.soft_reset()

In [ ]:
%serialconnect to --port="COM5"

%sendtofile /picotools.py --source picotools.py

import picotools as pico

import numpy as np
import matplotlib.pyplot as plt

import machine


## I3.1 DAC - Accuracy

The function in the cell below performs a sweep over all values of `DACA` and measures it with all three ADCs to check the accuracy of the measurement.

In [ ]:
ref, Ain0, Ain1, Ain2, err_out_0, err_out_1, err_out_2 = pico.test_DAC_A(NUM_SAMPLES = 100, gain0=1, gain1=0.3333)

In [ ]:
# Plot the measured DACA output vs Expected DACA output
plt.plot(ref, Ain0, label='AMP0')
plt.plot(ref, Ain1, label='AMP1')
plt.plot(ref, Ain2, label='Ain2')
plt.xlabel("Expected value from the DACA output [V]")
plt.ylabel("DACA measured output [V]")
plt.legend()

In [ ]:
# Plot the DACA Error vs DACA expected output
plt.plot(ref, err_out_0*1e3, label='AMP0')
plt.plot(ref, err_out_1*1e3, label='AMP1')
plt.plot(ref, err_out_2*1e3, label='Ain2')
plt.xlabel("Expected value from the DACA output [V]")
plt.ylabel("DACA measured error [mV]")
plt.legend()

1. Make a note of the magnitude of the error. **Will it affect your Voltammetry measurements?**
2. **Does it change when you disconnect -12V from Cria?** Run the measurements in the section with and without this connection.

In [ ]:
# Notes

###  I3.2 Resolution

In this section, you will experimentally find the resolution of the DAC. Use the same setup as before.

The provided test function: `pico.test_dac_resolution` measures a voltage ramp from `DACA` over very fine steps of `step_mV=0.1mV` over a range of `span_mV=10mV` using the `ADC0` via amplifier `AMP0`.

In the first attempt, `DACA` is set to a desired value, and it is measured only once at each step. 

1. Run the code below

In [ ]:
machine.soft_reset()

In [ ]:
%serialconnect to --port="COM5"

%sendtofile /picotools.py --source picotools.py

import picotools as pico

import numpy as np
import matplotlib.pyplot as plt
import machine

In [ ]:
voltages, ADC0, ADC0_errors, mean_error = pico.test_dac_resolution(adc=pico.adc0,gain=1,step_mV=0.1,set_initial_voltage_mV=2500,span_mV=10, N_iter=1)

plt.plot(voltages, ADC0, label="ADC0 Measurement")
plt.xlabel("DAC Voltage (mV)")
plt.ylabel("ADC Reading (mV)")
plt.title("DAC Voltage vs ADC Reading from a single measurement")
plt.legend()
plt.grid(True)

2. Have a look at the value of the *Absolute mean error* printed above the plot. Does it look familiar? 

Most likely it's similar to the ADC noise and DAC accuracy that you measured before. Confirm with the statistics plot below.

In [ ]:
plt.plot(voltages, ADC0_errors, label="ADC0 Measurement")
plt.xlabel("DAC Voltage (mV)")
plt.ylabel("ADC Reading Error (mV)")
plt.title("DAC Voltage vs ADC Reading Error from a single measurement")
plt.legend()
plt.grid(True)

We must therefore average over several measurements to resolve the finer steps. It turns out, that we must average a lot! 

3. In the next cell, each step is measured 4000 times! So, **it will take a few seconds to complete the measurement**, but be patient - It's worth it!

    > If by any chance, this large number of iterations causes a problem with Pico, you just learned why we need all those cumbersome *save-and-reset* procedures. <br>
    > Try lowering the `N_iter` to 1000 or so and reset Pico to resolve this issue. Then, re-run this entire section.

In [ ]:
voltages, ADC0, ADC0_errors, noise = pico.test_dac_resolution(adc=pico.adc0,gain=1,step_mV=0.1,set_initial_voltage_mV=2500,span_mV=10, N_iter=4000)

plt.plot(voltages, ADC0, label="ADC0 Measurement")
plt.xlabel("DAC Voltage (mV)")
plt.ylabel("ADC Reading (mV)")
plt.title("DAC Voltage vs ADC Reading from averaged measurement")
plt.legend()
plt.grid(True)

If everything went well, and you see *a staircase*, you should be able to deduce the DAC's resolution from the plot!

And you can also confirm it from the amplitude of the error plotted below.

In [ ]:
plt.plot(voltages, ADC0_errors, label="ADC0 Measurement")
plt.xlabel("DAC Voltage (mV)")
plt.ylabel("ADC Reading Error (mV)")
plt.title("DAC Voltage vs ADC Reading Error from averaged measurement")
plt.legend()
plt.grid(True)

### Conclusions
Think about the following questions and make some notes:

1.  What can you conclude about:
    - the accuracy of the DAC?
    - the noise when measuring no input and the noise when measuring the DAC signal?
2. What happens when you try to take smaller steps than the smallest detectable increment?

In the actual measurement, you won't be able to measure 4000 times at each step to average over the noise. You just learned that it takes a relatively long time, but you can afford to measure a few times at each step.

In the next section, you'll explore the effect of multiple measurements on timing of the experiment.

## Implement 4-Double ADC: DAC Assistant 

Implement 4 is **required** only for the **Double ADC Design**, and **optional** for the **Variable Offset Design.**

**Goal 1**: Calibrate your DAC Assistant and save the setting to a file<br>
**Goal 2**: Measure the the accuracy, and the effective resolution of the controls used in the Double ADC Design<br>
**Goal 3**: Practice setting attenuation `1:3` for positive and negative signals<br>

In the Alpaca Manual, you can find a formula for the output `DAC Assistant`:

$$
U^{\text{DAC Assistant}}_{OUT} = 5 \cdot (U^{\text{DAC Assistant}}_{+IN} - 2.048\text{V})
$$

In reality, the expected offset `2.048V` might have a significant error making your potentiostat unusable. 
Follow the procedure below to calibrate your DAC Assistant.

### I4.1 Double ADC Design: Calibration

1. Build the calibration setup as shown in the Fritzing below.

<details>
<summary><font color='darkred' size=4>💡</font> <b>Fritzing: DAC Assistant Calibration</b></summary><br>
<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/17_Fritzing_DAC_Assistant_Gain1.png" width=1000>
<br>
<em>DAC Assistant Calibration Setup</em>
</div><br> </details>


2. Run the code in the cells below to measure the accuracy of your DAC Assistant.

> This calibration procedure performs a voltage sweep from -3V to +3V and checks for the accuracy of the generated signal.


In [ ]:
machine.soft_reset()

In [ ]:
%serialconnect to --port="COM5"

%sendtofile /picotools.py --source picotools.py

import picotools as pico

import numpy as np
import matplotlib.pyplot as plt
import machine

In [ ]:
ref, Ain0, Ain1, Ain2, err_in, err_out = pico.test_DAC_Assistant()

In [ ]:
# Plot the DAC Assistant measured DAC Assistant output vs DAC A output
plt.plot((5 * (ref - 2.048)), Ain0+Ain1)
plt.xlabel("Expected value converted from the DAC_A output [V]")
plt.ylabel("DAC Assistant measured output [V]")

3. Do you observe a large offset error in the figure above?

In [ ]:
### Notes: 

4. Also, check the accuracy of `DACA`. Could it be causing such a large deviation?

In [ ]:
# Plot Error on the DAC_A (SET vs GET)
plt.plot(ref, 1e3*err_in)
plt.xlabel("DAC_A voltage [V]")
plt.ylabel("DAC_A output error [mV]")

In [ ]:
# Plot Error of the DAC_Assistant OUT (SET vs GET)
plt.plot((5 * (ref - 2.048)), err_out*1e3)
plt.xlabel("Expected DAC Assistant output [V]")
plt.ylabel("DAC Assistant output error [mV]")

It is rather unlikely that `DACA` inaccuracy leads to such a large error in the `DAC Assistant OUT`.
> We measured it before already, but keep in mind that `DACA` output error is *magnified five times* in the `DAC Assistant OUT`, so its effect here could be very significant. <br> <br>**This is one of the main causes of the *Double ADC Design* potentiostat's low performance in some experiments.**

At this point, we can only calibrate the offset, but the error that you see in the first from the two plots above is not likely to go away.

5. Run the calibration procedure in the cell below.

In [ ]:
calibrated_offset = pico.Calibrate_DAC_Assistant(1.5)

**If the calibration was successful, go ahead and SAVE THIS VALUE! ... and if it is not working, please report this problem to the TAs!**

Optionally, come up with a way to save this value in a file! 
> You can also run this calibration, on demand, without any modifications - as long as your setup matches the one required for this section.
> This won't be the case for long in this project, so it is recommended that you find a practical way to include this offset in your code.

Let's confirm that it worked by testing the accuracy again.

In [ ]:
ref, Ain0, Ain1, Ain2, err_in, err_out = pico.test_DAC_Assistant(offset=calibrated_offset)

In [ ]:
plt.plot((5 * (ref - calibrated_offset)), Ain0+Ain1)
plt.xlabel("DAC_A output [V]")
plt.ylabel("DAC Assistant measured output [V]")

You should see a straight line here!

6. Let's check the `DACA` error again. It should stay mostly the same, but shifted a bit.

In [ ]:
# plot Error on the DAC_A (SET vs GET)
plt.plot(ref, 1e3*err_in)
plt.xlabel("DAC_A voltage [V]")
plt.ylabel("DAC_A output error [mV]")

7. It's the moment of truth! 🥁 The accuracy of the calibrated DAC Assistant is....

In [ ]:
plt.plot((5 * (ref - calibrated_offset)), err_out*1e3)
plt.xlabel("DAC Assistant output [V]")
plt.ylabel("DAC Assistant output error [mV]")

8. How does it compare to the `DACA` output error?

In [ ]:
#Notes

## I4.2 Double ADC design: Resolution - DAC Assistant

Here, once again, you will test the resolution - This time, the one of the DAC Assistant. 

**The insights of this section are essential for finding the limits for parameters to run Cyclic and Squarewave Voltammetry with the Double ADC Design**

Use the same setup as in the section above and follow the steps of the familiar procedure.

In [ ]:
machine.soft_reset()

In [ ]:
%serialconnect to --port="COM5"

%sendtofile /picotools.py --source picotools.py

import picotools as pico

import numpy as np
import matplotlib.pyplot as plt
import machine

In [ ]:
calibrated_offset = pico.Calibrate_DAC_Assistant(1.5)

Let's first try with just one measurement at each step. 


In [ ]:
N_iter = 1
voltages, ADC0, ADC0_errors, mean_error = pico.test_dac_assistant_resolution(adc=pico.adc0,gain=1,step_mV=0.1,set_initial_voltage_mV=2000,span_mV=10, N_iter=N_iter, calibrated_offset=calibrated_offset)

plt.plot(voltages, ADC0, label="ADC0 Measurement")
plt.xlabel("DAC Assistant Voltage (mV)")
plt.ylabel("ADC Reading (mV)")
plt.title(f'DAC Assistant Voltage vs ADC Reading avg from {N_iter} measurement(s)')
plt.legend()
plt.grid(True)

2. Is the *Absolute mean error* printed above the plot similar as for DAC in [Link to I3.2 Resolution](#--i3.2-resolution)?

Let's get a better idea of it with the statistics plot:

In [ ]:
plt.plot(voltages, ADC0_errors, label="ADC0 Measurement")
plt.xlabel("DAC Assistant Voltage (mV)")
plt.ylabel("ADC Reading Error (mV)")
plt.title(f'DAC Assistant Voltage vs ADC Reading avg from {N_iter} measurement(s)')
plt.legend()
plt.grid(True)

Most likely, it is much bigger, which is expected. 

> Think about the gain factor in the `DAC Assistant OUT` formula. Could it be related to the magnitude of this error?

3. So, let's again average over many samples to remove the noise.

In [ ]:
N_iter = 5000
voltages, ADC0, ADC0_errors, mean_error = pico.test_dac_assistant_resolution(adc=pico.adc0,gain=1,step_mV=0.1,set_initial_voltage_mV=2000,span_mV=10, N_iter=N_iter, calibrated_offset=calibrated_offset)

plt.plot(voltages, ADC0, label="ADC0 Measurement")
plt.xlabel("DAC Assistant Voltage (mV)")
plt.ylabel("ADC Reading (mV)")
plt.title("DAC Assistant Voltage vs ADC Reading from averaged measurement")
plt.legend()
plt.grid(True)

Do you still see *a staircase*? 

In this case, the steps are most likely much bigger, and therefore, the resolution lower.

4. You can also confirm with the amplitude of the error plotted below.

In [ ]:
plt.plot(voltages, ADC0_errors, label="ADC0 Measurement")
plt.xlabel("DAC Voltage (mV)")
plt.ylabel("ADC Reading Error (mV)")
plt.title("DAC Voltage vs ADC Reading Error from averaged measurement")
plt.legend()
plt.grid(True)

### Conclude

1. What is therefore the smallest step for the `Ucell` increment in your Voltammetry measurements with the Double ADC Design?
2. And taking into considerations that in your measurements, you won't be able to average over thoursands of samples, but only a few. What is the realistic resolution?

In [ ]:
### Notes:

# Implement 5-Double ADC design: ADC Gain 1:3

**The section I5.1 is mandatory for Double ADC Design:** Learn how to implement attenuation 1:3 with Double ADC Design. 

### I5.1 Double ADC: Configure and test ADC Gain 1:3 

Implement the setup presented in the Fritzing below and run the test below to find out if you got it right!

<details><summary> <font color='darkred' size=4>💡</font> 
<b>Fritzing: Double ADC Design - Gain 1:3</b></summary><br>
<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/17_Fritzing_DAC_Assistant_Gain03.png" width=1000>
<br>
<em>DAC Test</em>
</div><br>
<br>
</details> <br>

In [ ]:
machine.soft_reset()

In [ ]:
%serialconnect to --port="COM5"

%sendtofile /picotools.py --source picotools.py

import picotools as pico

import numpy as np
import matplotlib.pyplot as plt
import machine

In [ ]:
calibrated_offset = 2.2826

In [ ]:
# GAIN (1:3)
ref, Ain0v, Ain1v, Ain2v, err_in, err_out = pico.test_DAC_Assistant(offset=calibrated_offset, gain0=0.333, gain1=-0.333)

In [ ]:
# Plot the DAC Assistant measured DAC Assistant output vs DAC A output
plt.plot((5 * (ref - calibrated_offset)), Ain0v+Ain1v)
plt.xlabel("Expected value converted from the DAC_A output [V]")
plt.ylabel("DAC Assistant measured output [V]")

In [ ]:
### Notes:

# Implement 6: Write Helper Functions

Write some helper functions to make your further steps easier. Here are some ideas:

1. Function to calculate the detectable `Icell` range based on the `Ucell`, `Rf` and the values of attenuation. Its form will vary per design.
2. Function to calculate the cutoff frequency of the *tamed* integrator.
3. Your own ideas!

💡 Hint: the following functions could be useful for the Variable Offset Design only
4. Function for using the Alpaca's Relay for switching between two different `Rf` resistors.
5. Function for including the use of DAC-B


In [ ]:
# Task for the students!

# Compare and Conclude

**To check-off with the TA:**
From DAC-accuracy:
1. Make a note of the magnitude of the error. **Will it affect your Voltammetry measurements?**
2. **Does it change when you disconnect -12V from Cria?** Run the measurements in the section with and without this connection.
3. (for the Double ADC Design): discuss the outcome and necessity of the DAC assistant calibration
4. (for the Variable Offset Design): discuss which additional picotools functions you will/did write for use of DAC-B

**Mandatory to think about (for the report):**
>1. What can you conclude about the smallest detectable increment of the:
>   - DAC Assistant?
>   - DAC A measured directly?
>2. What happens when you try to take smaller steps than the smallest detectable increment?
>3. How does it relate to the resolution of the ADCs?
>4. Can you already predict what will be the lowest effective `Ucell` increment for Voltammetry with your design?

